In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import zipfile
import os

# Đường dẫn đến file zip đã tải lên
zip_path = "/content/data.zip"

# Thư mục để giải nén
extract_dir = "data_out"

# Giải nén file zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Hiển thị danh sách các file sau khi giải nén
print("Các file sau khi giải nén:")
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(os.path.join(root, file))


In [ ]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

raw_folder = "/content/data"
print("Danh sách các thư mục con trong 'raw_folder':", listdir(raw_folder))

def save_data(raw_folder=raw_folder):
    dest_size = (128, 128)
    print("Bắt đầu xử lý ảnh...")

    pixels = []
    labels = []

    # Lặp qua các folder con trong thư mục raw
    for folder in listdir(raw_folder):
        folder_path = os.path.join(raw_folder, folder)
        if os.path.isdir(folder_path) and folder != '.DS_Store':
            print("Folder=", folder)
            # Lặp qua các file trong từng thư mục chứa các em
            for file in listdir(folder_path):
                if file != '.DS_Store':
                    file_path = os.path.join(folder_path, file)
                    print("File=", file)
                    pixels.append(cv2.resize(cv2.imread(file_path), dsize=(128,128)))
                    labels.append(folder)

    pixels = np.array(pixels)
    labels = np.array(labels)#.reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    file = open('pix.data', 'wb')
    # dump information to that file
    pickle.dump((pixels,labels), file)
    # close the file
    file.close()

    return

def load_data():
    file = open('pix.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    print(pixels.shape)
    print(labels.shape)

    return pixels, labels

save_data()
X,y = load_data()
#random.shuffle(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

print(X_train.shape)
print(y_train.shape)

def get_model():
    model_resnet50v2 = ResNet50V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    # Đóng băng các layer
    for layer in model_resnet50v2.layers:
        layer.trainable = False

    # Tạo model
    input = Input(shape=(128, 128, 3), name='image_input')
    output_resnet50v2 = model_resnet50v2(input)

    # Thêm các layer FC và Dropout
    x = Flatten(name='flatten')(output_resnet50v2)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.3)(x)
    x = Dense(4, activation='softmax', name='predictions')(x)

    # Compile
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return my_model

resnet_model = get_model()

filepath = "weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.2,
                         rescale=1./255,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         horizontal_flip=True,
                         brightness_range=[0.2, 1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

# Fit the model
resnet50_hist = resnet_model.fit(aug.flow(X_train, y_train, batch_size=64), epochs=50, validation_data=aug_val.flow(X_test, y_test, batch_size=64), callbacks=callbacks_list)

# Save the model
resnet_model.save("resnet_model.h5")

def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(range(1, len(model_history.history[acc]) + 1), model_history.history[acc])
    axs[0].plot(range(1, len(model_history.history[val_acc]) + 1), model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_history.history[acc]) + 1), len(model_history.history[acc]) / 10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
    axs[1].plot(range(1, len(model_history.history['val_loss']) + 1), model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(model_history.history['loss']) + 1), len(model_history.history['loss']) / 10)
    axs[1].legend(['train', 'val'], loc='best')
    #plt.show()
    plt.savefig('roc.png')